Classify pigmented skin lesions dermatoscopic images from HAM10k https://www.nature.com/articles/sdata2018161 into 7 diagnosis


In [38]:
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import load_img, img_to_array

from tensorflow.keras import models, regularizers, layers, optimizers, losses, metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

import os

In [42]:
# Convoluted Base MODEL

conv_base = ResNet50(weights='imagenet',
include_top=False,
input_shape=(224, 224, 3))

print(conv_base.summary())

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_3[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                       

In [43]:
# MODEL

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(64, activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='sigmoid'))

print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten_2 (Flatten)         (None, 100352)            0         
                                                                 
 dropout_4 (Dropout)         (None, 100352)            0         
                                                                 
 dense_4 (Dense)             (None, 64)                6422592   
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 2)                 130       
                                                                 
Total params: 30010434 (114.48 MB)
Trainable params: 2

In [44]:
# Make the conv_base NOT trainable:

for layer in conv_base.layers[:]:
   layer.trainable = False

print('conv_base is now NOT trainable')

conv_base is now NOT trainable


In [45]:
for i, layer in enumerate(conv_base.layers):
   print(i, layer.name, layer.trainable)

0 input_3 False
1 conv1_pad False
2 conv1_conv False
3 conv1_bn False
4 conv1_relu False
5 pool1_pad False
6 pool1_pool False
7 conv2_block1_1_conv False
8 conv2_block1_1_bn False
9 conv2_block1_1_relu False
10 conv2_block1_2_conv False
11 conv2_block1_2_bn False
12 conv2_block1_2_relu False
13 conv2_block1_0_conv False
14 conv2_block1_3_conv False
15 conv2_block1_0_bn False
16 conv2_block1_3_bn False
17 conv2_block1_add False
18 conv2_block1_out False
19 conv2_block2_1_conv False
20 conv2_block2_1_bn False
21 conv2_block2_1_relu False
22 conv2_block2_2_conv False
23 conv2_block2_2_bn False
24 conv2_block2_2_relu False
25 conv2_block2_3_conv False
26 conv2_block2_3_bn False
27 conv2_block2_add False
28 conv2_block2_out False
29 conv2_block3_1_conv False
30 conv2_block3_1_bn False
31 conv2_block3_1_relu False
32 conv2_block3_2_conv False
33 conv2_block3_2_bn False
34 conv2_block3_2_relu False
35 conv2_block3_3_conv False
36 conv2_block3_3_bn False
37 conv2_block3_add False
38 conv2_bloc

In [46]:
# Compile frozen conv_base + my top layer

model.compile(optimizer=optimizers.Adam(),
              loss='binary_crossentropy',
              metrics=['accuracy'])

print("model compiled")
print(model.summary())

model compiled
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten_2 (Flatten)         (None, 100352)            0         
                                                                 
 dropout_4 (Dropout)         (None, 100352)            0         
                                                                 
 dense_4 (Dense)             (None, 64)                6422592   
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 2)                 130       
                                                                 
Total params: 30010434 (114.48 MB)
Trai

In [47]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch_size = 32  # Define the batch size
desired_size = (224, 224)

# train_datagen = ImageDataGenerator(
#     rescale=1./255,
#     validation_split=0.2  # Using 20% of data for validation
# )

train_datagen = ImageDataGenerator(
     rescale=1./255,
    validation_split=0.2
)


# Adjust the directory paths to the preprocessed dataset
train_generator = train_datagen.flow_from_directory(
    directory='dataset',
    target_size=desired_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

validation_generator = train_datagen.flow_from_directory(
    directory='dataset',
    target_size=desired_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle= False
)


Found 7246 images belonging to 2 classes.
Found 1811 images belonging to 2 classes.


In [48]:
print(train_generator.class_indices)
print(validation_generator.class_indices)
# print(test_generator.class_indices)

{'bad': 0, 'good': 1}
{'bad': 0, 'good': 1}


It’s necessary to freeze the convolution base of the conv base in order to
be able to train a randomly initialized classifier on top. For the same reason, it’s only
possible to fine-tune the top layers of the convolutional base **once the classifier on top
has already been trained**. If the classifier isn’t already trained, then the error signal
propagating through the network during training will be too large, and the representations
previously learned by the layers being fine-tuned will be destroyed

**Below, first train with no limit to lr - with conv_base frozen - only  my top layers**

**Then, unfreeze last model conv block , recompile and train all with LOW lr=1e-5**

In [49]:
# Short training ONLY my top layers 
#... so the conv_base weights will not be destroyed by the random intialization of the new weights

history = model.fit(
    train_generator,
    epochs=3,  # Adjust as necessary
    validation_data=validation_generator,)

Epoch 1/3
227/227 [==============================] - 356s 2s/step - loss: 0.6507 - accuracy: 0.7621 - val_loss: 0.5265 - val_accuracy: 0.7642
Epoch 2/3
227/227 [==============================] - 327s 1s/step - loss: 0.5773 - accuracy: 0.7643 - val_loss: 0.5208 - val_accuracy: 0.7642
Epoch 3/3
227/227 [==============================] - 325s 1s/step - loss: 0.5572 - accuracy: 0.7643 - val_loss: 0.5420 - val_accuracy: 0.7642


In [50]:
# Make last block of the conv_base trainable:

for layer in conv_base.layers[:165]:
   layer.trainable = False
for layer in conv_base.layers[165:]:
   layer.trainable = True

print('Last block of the conv_base is now trainable')

Last block of the conv_base is now trainable


In [51]:
for i, layer in enumerate(conv_base.layers):
   print(i, layer.name, layer.trainable)

0 input_3 False
1 conv1_pad False
2 conv1_conv False
3 conv1_bn False
4 conv1_relu False
5 pool1_pad False
6 pool1_pool False
7 conv2_block1_1_conv False
8 conv2_block1_1_bn False
9 conv2_block1_1_relu False
10 conv2_block1_2_conv False
11 conv2_block1_2_bn False
12 conv2_block1_2_relu False
13 conv2_block1_0_conv False
14 conv2_block1_3_conv False
15 conv2_block1_0_bn False
16 conv2_block1_3_bn False
17 conv2_block1_add False
18 conv2_block1_out False
19 conv2_block2_1_conv False
20 conv2_block2_1_bn False
21 conv2_block2_1_relu False
22 conv2_block2_2_conv False
23 conv2_block2_2_bn False
24 conv2_block2_2_relu False
25 conv2_block2_3_conv False
26 conv2_block2_3_bn False
27 conv2_block2_add False
28 conv2_block2_out False
29 conv2_block3_1_conv False
30 conv2_block3_1_bn False
31 conv2_block3_1_relu False
32 conv2_block3_2_conv False
33 conv2_block3_2_bn False
34 conv2_block3_2_relu False
35 conv2_block3_3_conv False
36 conv2_block3_3_bn False
37 conv2_block3_add False
38 conv2_bloc

In [52]:
from tensorflow.keras.optimizers import Adam

# Compile model
initial_learning_rate = 1e-5
model.compile(optimizer=Adam(learning_rate=initial_learning_rate),
              loss='binary_crossentropy',
              metrics=['accuracy'])

print("model compiled")
print(model.summary())

model compiled
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten_2 (Flatten)         (None, 100352)            0         
                                                                 
 dropout_4 (Dropout)         (None, 100352)            0         
                                                                 
 dense_4 (Dense)             (None, 64)                6422592   
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 2)                 130       
                                                                 
Total params: 30010434 (114.48 MB)
Trai

In [61]:
# Long training with fine tuning

history = model.fit(train_generator,
                              epochs=10,  # Adjust as necessary
                              validation_data=validation_generator,)

model.save('ResNet50FineTune.keras')
print("ResNet50FineTune.keras was saved")

Epoch 1/10
227/227 [==============================] - 452s 2s/step - loss: 0.5210 - accuracy: 0.7643 - val_loss: 0.5029 - val_accuracy: 0.7642
Epoch 2/10
227/227 [==============================] - 437s 2s/step - loss: 0.5156 - accuracy: 0.7643 - val_loss: 0.4931 - val_accuracy: 0.7642
Epoch 3/10
227/227 [==============================] - 438s 2s/step - loss: 0.5149 - accuracy: 0.7643 - val_loss: 0.4826 - val_accuracy: 0.7642
Epoch 4/10
227/227 [==============================] - 441s 2s/step - loss: 0.5136 - accuracy: 0.7643 - val_loss: 0.5179 - val_accuracy: 0.7642
Epoch 5/10
227/227 [==============================] - 441s 2s/step - loss: 0.5090 - accuracy: 0.7643 - val_loss: 0.4895 - val_accuracy: 0.7642
Epoch 6/10
227/227 [==============================] - 436s 2s/step - loss: 0.5090 - accuracy: 0.7643 - val_loss: 0.5082 - val_accuracy: 0.7642
Epoch 7/10
227/227 [==============================] - 404s 2s/step - loss: 0.5123 - accuracy: 0.7643 - val_loss: 0.4987 - val_accuracy: 0.7642

In [58]:
from tensorflow.keras.models import load_model

# Assuming your model is saved under 'best_model.keras'
model = load_model('MelanomaResNet50FineTune.keras')

In [59]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch_size = 32  # Adjust to your preference
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    directory='test_dataset_processed',
    target_size=(224, 224),  # Ensure this matches your model's expected input
    batch_size=batch_size,
    class_mode='categorical',  # or 'binary' for binary classification
    shuffle=False  # Important for confusion matrix
)

Found 1090 images belonging to 2 classes.


In [60]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

35/35 [==============================] - 50s 1s/step - loss: 0.5162 - accuracy: 0.7275
Test Loss: 0.5161858797073364
Test Accuracy: 0.7275229096412659
